In [1]:
from datetime import date
import pandas as pd
import os
import win32com.client
from openpyxl import load_workbook
from openpyxl.styles import Alignment, Font, Border, Side

import warnings
warnings.simplefilter(action="ignore", category=Warning)

In [6]:
year_now = date.today().year  # текущий год
# имя файла с учетом текущего года
# file = "//Server/otk/1 ГАРАНТИЯ на сервере/" + str(year_now) + "-2019_ЖУРНАЛ УЧЁТА.xlsm"
file = f"{str(year_now)}-2019_ЖУРНАЛ УЧЁТА.xlsm"


df = pd.read_excel(
            file,
            sheet_name=str(year_now),
            usecols=[
                "Месяц регистрации",
                "Дата поступления сообщения в ОТК",
                "Период выявления дефекта (отказа)",
                "Наименование изделия",
                "Обозначение изделия",
                "Заводской номер изделия",
                "Дата изготовления изделия",
                "Пробег, наработка",
                "Заявленный дефект изделия",
                "Количество предъявленных изделий"
            ],
            header=1,
        )

In [7]:
df.rename(
    columns={
        "Период выявления дефекта (отказа)": "Период выявления",
        "Количество предъявленных изделий": "Количество",
        "Заявленный дефект изделия": "Заявленный дефект"
    },
    inplace=True
)

In [8]:
# Номер строки датафрейма (индекс строки) делаем как в базе данных
df.index = df.index + 3

In [9]:
# Удаляем строки в которых нет информации
df_c = df.dropna(subset=["Период выявления"])

In [10]:
# В обозначении изделий убираем перенос строк
df_c["Обозначение изделия"] = df_c["Обозначение изделия"].apply(lambda x: x.split("\n")[0] if "\n" in x else x)

In [11]:
# Изменяем тип данных в столбце "Количество"
df_c["Количество"] = df_c["Количество"].astype('int16')

In [12]:
# Заменяем отсутствующие значения в столбце "Заявленный дефект" на значение "неизвестно"
df_c["Заявленный дефект"].fillna("неизвестно", inplace=True)

In [13]:
df_c

,Месяц регистрации,Дата поступления сообщения в ОТК,Период выявления,Наименование изделия,Обозначение изделия,Заводской номер изделия,Дата изготовления изделия,"Пробег, наработка",Заявленный дефект,Количество
3,январь,2025-01-08,ММЗ - АСП,водяной насос,260-1307116-15,б/н,б/д,ПСИ,"течь ОЖ, раковина корпуса насоса",1
4,январь,NaT,запчасть,компрессор,5336-3509012,2478,03.24,без наработки,низкая производительность\n(около 7 атм.),1
5,январь,NaT,запчасть,компрессор,ПК 310,963,04.24,NaN,перепускает в систему охлаждения,1
6,январь,NaT,ММЗ - эксплуатация,водяной насос,240-1307010-А1,б/н,NaN,NaN,течь,1
7,январь,NaT,ММЗ - эксплуатация,водяной насос,240-1307010-А1,б/н,NaN,NaN,течь,1
...,...,...,...,...,...,...,...,...,...,...
202,январь,2025-01-23,ЯМЗ - эксплуатация,компрессор,ПК 225,00114,04.22,2372 км,недостаточно производительности,1
203,январь,2025-01-23,ЯМЗ - эксплуатация,водяной насос,КБПА 451363.21,NaN,NaN,36642 км,течь,1
204,январь,2025-01-23,ЯМЗ - эксплуатация,водяной насос,КБПА 451363.21,NaN,NaN,66240 км,течь,1
205,январь,2025-01-23,ЯМЗ - эксплуатация,водяной насос,КБПА 451363.21,NaN,NaN,66365 км,течь,1


In [14]:
df_c.info()

<class 'pandas.core.frame.DataFrame'>
Index: 204 entries, 3 to 206
Data columns (total 10 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   Месяц регистрации                 204 non-null    object        
 1   Дата поступления сообщения в ОТК  104 non-null    datetime64[ns]
 2   Период выявления                  204 non-null    object        
 3   Наименование изделия              204 non-null    object        
 4   Обозначение изделия               204 non-null    object        
 5   Заводской номер изделия           162 non-null    object        
 6   Дата изготовления изделия         36 non-null     object        
 7   Пробег, наработка                 97 non-null     object        
 8   Заявленный дефект                 204 non-null    object        
 9   Количество                        204 non-null    int16         
dtypes: datetime64[ns](1), int16(1), object(8)
memory usage:

In [174]:
# Номер последней строки базы данных с информацией по рекламациям
df_c.index[-1]

np.int64(171)

In [175]:
# Диапазон строк базы данных по которым формируется отчет: 3 - 171

In [ ]:
# Формируем отчет за период ... df_res = df_c.loc[3:].groupby(
df_res = df_c.groupby(
    [
        "Период выявления",
        "Наименование изделия",
        "Обозначение изделия",
        "Заявленный дефект"
    ]
)["Количество"].sum().to_frame()  # agg('count').to_frame().rename(columns={"Месяц регистрации": "Количество"})

In [177]:
df_res

Количество
Период выявления           Наименование изделия Обозначение изделия Заявленный дефект                                             
ММЗ - АСП                  водяной насос        245-1307010-А1-11   неизвестно                                                   1
                                                260-1307116-05      неизвестно                                                   1
                                                260-1307116-15      течь ОЖ, раковина корпуса насоса                             1
                                                3LD-1307010-Б       неизвестно                                                   3
                           коромысло клапана    245-1007212         заклинил гидрокомпенсатор                                   12
                                                                    стружка в масляном канале                                   92
                           масляный насос       3LD-1403010         посторонние частицы, загрязнение внутренней пол...         115
                           турбокомпрессор      ТКР 6-03.10         нет наддува                                                118
ММЗ - эксплуатация         водяной насос        240-1307010-А1      течь                                                        12
                                                240-1307010-А1-01   течь                                                         1
                                                245-1307010-А1-11   течь                                                         8
                                                260-1307116-02      течь                                                        28
                                                260-1307116-05      течь                                                         4
                                                260-1307116-06      течь                                                         4
                                                260-1307116-15      течь                                                        10
                           компрессор           А29.05.000-06-БЗА   неизвестно                                                   1
                                                А29.05.000-БЗА      неизвестно                                                  10
                           турбокомпрессор      ТКР 6-01.01         неизвестно                                                   2
                                                ТКР 7-02.08         неизвестно                                                   1
Ростсельмаш - АСП          компрессор           5336-3509012-К181   течь                                                         1
Ростсельмаш - эксплуатация компрессор           5336-3509012-К181   износ компонента                                             1
                                                                    течь                                                         2
УРАЛ - эксплуатация        компрессор           5336-3509012        заклинивание                                                 1
                                                                    неизвестно                                                   3
                                                                    трещина на корпусе                                           1
                                                5336-3509012-10     перестал качать                                              1
ЯМЗ - АСП                  компрессор           ПК 306-03           течь                                                         1
ЯМЗ - эксплуатация         водяной насос        КБПА 451363.21      слетел шкив с вала                                           1
                                                                    течь                                                        56
                           масляный насос       КБПА 451412

In [179]:
# записываем в файл TXT
with open(f"//Server/otk/Support_files_не_удалять!!!/Справка по дефектам за период.txt", "w", encoding="utf-8") as f:
    print(f"\n\n\tСправка по поступившим рекламациям за период 27.11.24 - 03.12.24", file=f)
    f.write(df_res.to_string())

print("Справка в файл TXT записана")

Справка в файл TXT записана


In [180]:
# Файл Excel в который будет записываться справка
file_excel = f"//Server/otk/ПРОТОКОЛЫ совещаний по качеству/{year_now}/Справка по дефектам за период.xlsx"

In [181]:
# записываем в файл Excel
df_res.to_excel(file_excel)
print("Справка в файл Excel записана")

Справка в файл Excel записана


In [182]:
# Редактируем стили и выравнивание в файле Excel справки

wb = load_workbook(file_excel)  # открываем файл Excel
sheet = wb["Sheet1"]  # делаем активным Лист "Sheet1"

# вставляем столбец в позицию 0
sheet.insert_cols(0)

# задаем высоту строки 1 (с названиями столбцов)
sheet.row_dimensions[1].height = 15

# задаем ширину столбцов B, C, D, E
sheet.column_dimensions["B"].width = 23
sheet.column_dimensions["C"].width = 20
sheet.column_dimensions["D"].width = 20
sheet.column_dimensions["E"].width = 23
sheet.column_dimensions["F"].width = 10

# колонки таблицы
cols = "B", "C", "D", "E", "F"

# определяем количество строк в таблице (длина итогового датафрейма)
len_table = len(df_res)

# циклом по столбцам таблицы
for i in cols:
    # активируем перенос текста в ячейках B1, C1, D1, E1 (с названиями столбцов) и выравниваем по центру
    sheet[f"{i + str(1)}"].alignment = Alignment(wrap_text=True, horizontal="center", vertical="center")
    # циклом по строкам таблицы
    for j in range(1, len_table + 2):
        # задаем стиль границы - тонкая линия и цвет черный
        thins = Side(border_style="thin", color="000000")
        # применяем заданный стиль границы к верхней, нижней, левой и правой границе ячеек по циклу
        sheet[f"{i + str(j)}"].border = Border(top=thins, bottom=thins, left=thins, right=thins)
        # изменяем шрифт в ячейках с жирного на обычный
        sheet[f"{i + str(j)}"].font = Font(name='Times New Roman', size=10, bold=False)

for i in ("B", "C", "D", "E"):
    for j in range(2, len_table + 2):
        # выравниваем текст в ячейках "B", "C", "D", "E" по левому краю по верху с переносом текста
        sheet[f"{i + str(j)}"].alignment = Alignment(wrap_text=True, horizontal="left", vertical="top")
for j in range(2, len_table + 2):
    # выравниваем текст в ячейке "F" по центру
        sheet[f"F{str(j)}"].alignment = Alignment(horizontal="center", vertical="center")

# сохраняем изменения
wb.save(file_excel)
print("Отредактированный файл Excel со справкой записан")

Отредактированный файл Excel со справкой записан


In [183]:
# Создаем файл Word со справкой
# Копируем таблицу из файла Excel и вставляем в файл Word

# pip install pywin32
import win32com.client

# Создаем объект Excel
excel = win32com.client.Dispatch("Excel.Application")
# excel.Visible = True  # Делает Excel видимым

# Указываем файл Excel, который хотим открыть
# file_excel_path = "D:\MyRepositories\EDUCATION\Python тренажер\example.xlsx"   # тренажер
workbook = excel.Workbooks.Open(file_excel)
worksheet = workbook.Sheets(1)  # Выбираем активный лист
# Определяем диапазон данных в столбцах A:D
data_range = worksheet.Range("A1").CurrentRegion

data_range.Copy()  # Копируем диапазон данных
excel.Quit()  # Закрываем Excel

print("Данные из файла Excel скопированы в буфер")

# Создаем объект Word
word = win32com.client.Dispatch("Word.Application")

# Указываем файл, который хотим открыть
file_doc = f"//Server/otk/ПРОТОКОЛЫ совещаний по качеству/{year_now}/Справка по дефектам за период.docx"
doc = word.Documents.Open(file_doc)

# Вставляем текст в первую строку документа
# word.Selection.HomeKey(win32com.client.constants.wdStory)
word.Selection.TypeText("Справка по количеству рекламаций за период\n\n")

word.Selection.Paste()  # Вставляем данные из буфера в документ

doc.Save()  # Сохраняем документ
word.Quit()  # Закрываем Word

print("Данные из буфера записаны в файл Word")
print("Создание справки в файлы Excel и Word завершено")

Данные из файла Excel скопированы в буфер
Данные из буфера записаны в файл Word
Создание справки в файлы Excel и Word завершено
